# Compreendendo a Forma das RNN's

Neste notebook, vamos compreender a forma (**shape**) dos dados para que possamos trabalhar adequadamente com as RNN's.

Algumas nomenclaturas importantes:

- **N** = número de amostras
- **T** = comprimento da sequência
- **D** = número de features de input
- **M** = número de hidden units
- **K** = número de output units

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

Criamos dados sintéticos para o nosso experimento:

In [2]:
N = 1
T = 10
D = 3
M = 5
K = 2
X = np.random.randn(N, T, D)
print(X)

[[[-0.00943505 -1.32602294  0.44338797]
  [ 0.83251681  0.70938055  1.37799121]
  [ 0.85904206  0.69557623  2.63294845]
  [ 1.15123266  1.82088775  0.3270658 ]
  [ 1.50675687 -2.3714614   0.40914567]
  [-0.65683853  1.19596887 -0.08413552]
  [ 0.40377389  1.61672659  1.43369377]
  [ 0.10751359 -0.73816713  0.29595958]
  [-0.51243643 -0.61586604  0.12251861]
  [-1.56424461  0.13736604 -0.39195131]]]


Construímos a RNN:

In [3]:
class SimpleRNN(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(SimpleRNN, self).__init__()
        self.D = n_inputs
        self.M = n_hidden
        self.K = n_outputs
        self.rnn = nn.RNN(
            input_size=self.D,
            hidden_size=self.M,
            nonlinearity='tanh',
            batch_first=True)
        self.fc = nn.Linear(self.M, self.K)

    def forward(self, X):
        # initial hidden states
        h0 = torch.zeros(1, X.size(0), self.M)

        # get RNN unit output
        out, _ = self.rnn(X, h0)

        # we only want h(T) at the final time step
        # out = self.fc(out[:, -1, :])
        out = self.fc(out)
        return out

Instanciamos o modelo:

In [4]:
model = SimpleRNN(n_inputs=D, n_hidden=M, n_outputs=K)

Obtemos o *output*:

In [5]:
inputs = torch.from_numpy(X.astype(np.float32))
out = model(inputs)
print(out)

tensor([[[-0.2869,  0.3848],
         [-0.4218,  0.2541],
         [-0.4719,  0.2888],
         [-0.1923, -0.0218],
         [-0.0209,  0.3298],
         [-0.2076,  0.0873],
         [-0.3614,  0.0780],
         [-0.1170,  0.0927],
         [-0.0999,  0.1863],
         [-0.0784, -0.0010]]], grad_fn=<AddBackward0>)


Acessamos o atributo **shape**:

In [6]:
print(out.shape)

torch.Size([1, 10, 2])


Salvamos o *output* para uso posterior:

In [7]:
Yhats_torch = out.detach().numpy()
print(Yhats_torch)

[[[-0.2868827   0.38481712]
  [-0.4218371   0.25413838]
  [-0.47191644  0.28883094]
  [-0.19227447 -0.02184061]
  [-0.02091083  0.32980207]
  [-0.20760347  0.08731602]
  [-0.36137658  0.07796021]
  [-0.11702731  0.0926653 ]
  [-0.09993263  0.18629757]
  [-0.07844485 -0.0009826 ]]]


Obtemos os parâmetros da RNN:

In [10]:
W_xh, W_hh, b_xh, b_hh = model.rnn.parameters()

print(W_xh.shape)
print(W_xh)

torch.Size([5, 3])
Parameter containing:
tensor([[-0.1637, -0.3728, -0.2261],
        [-0.0897, -0.3706, -0.1556],
        [ 0.3651, -0.3630, -0.1952],
        [-0.0349, -0.2168,  0.2665],
        [-0.0537,  0.0343, -0.0485]], requires_grad=True)


Convertemos os parâmetros para NumPy:

In [11]:
W_xh = W_xh.data.numpy()
b_xh = b_xh.data.numpy()
W_hh = W_hh.data.numpy()
b_hh = b_hh.data.numpy()

Vejamos se está correto:

In [12]:
W_xh.shape, b_xh.shape, W_hh.shape, b_hh.shape

((5, 3), (5,), (5, 5), (5,))

Também iremos obter os parâmetros da Fully Connected Layer (FC Layer):

In [13]:
Wo, bo = model.fc.parameters()

E convertemos eles para NumPy:

In [14]:
Wo = Wo.data.numpy()
bo = bo.data.numpy()
Wo.shape, bo.shape

((2, 5), (2,))

Vejamos se podemos replicar o *output* com os parâmetros:

In [15]:
h_last = np.zeros(M) # initial hidden state
x = X[0] # the one and only sample
Yhats = np.zeros((T, K)) # where we store the outputs

for t in range(T):
    h = np.tanh(x[t].dot(W_xh.T) + b_xh + h_last.dot(W_hh.T) + b_hh)
    y = h.dot(Wo.T) + bo # we only care about this value on the last iteration
    Yhats[t] = y

    # important: assign h to h_last
    h_last = h

Imprimimos o *output* final:

In [16]:
print(Yhats)

[[-0.28688266  0.38481712]
 [-0.42183714  0.25413839]
 [-0.47191646  0.28883093]
 [-0.19227447 -0.02184063]
 [-0.02091083  0.32980204]
 [-0.20760349  0.087316  ]
 [-0.36137659  0.07796023]
 [-0.1170273   0.09266529]
 [-0.09993263  0.18629755]
 [-0.07844485 -0.0009826 ]]


Por fim, checamos se está correto:

In [17]:
np.allclose(Yhats, Yhats_torch)

True